In [1]:
import pandas as pd
import sqlite3, matplotlib
from sqlalchemy import create_engine
import re

Read data from CSV

In [2]:
# read the Park Data
df_pr = pd.read_csv('parks.csv', encoding='utf-8')
# read the Species Data
df_sp = pd.read_csv('species.csv', encoding='utf-8')

/Users/minglei/anaconda3/envs/PythonData/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df_pr.head()

,Park Code,Park Name,State,Acres,Latitude,Longitude
0,ACAD,Acadia National Park,ME,47390,44.35,-68.21
1,ARCH,Arches National Park,UT,76519,38.68,-109.57
2,BADL,Badlands National Park,SD,242756,43.75,-102.50
3,BIBE,Big Bend National Park,TX,801163,29.25,-103.25
4,BISC,Biscayne National Park,FL,172924,25.65,-80.08


In [4]:
df_sp.head(100)

,Species ID,Park Name,Category,Order,Family,Scientific Name,Common Names,Record Status,Occurrence,Nativeness,Abundance,Seasonality,Conservation Status,Unnamed: 13
0,ACAD-1000,Acadia National Park,Mammal,Artiodactyla,Cervidae,Alces alces,Moose,Approved,Present,Native,Rare,Resident,NaN,NaN
1,ACAD-1001,Acadia National Park,Mammal,Artiodactyla,Cervidae,Odocoileus virginianus,"Northern White-Tailed Deer, Virginia Deer, Whi...",Approved,Present,Native,Abundant,NaN,NaN,NaN
2,ACAD-1002,Acadia National Park,Mammal,Carnivora,Canidae,Canis latrans,"Coyote, Eastern Coyote",Approved,Present,Not Native,Common,NaN,Species of Concern,NaN
3,ACAD-1003,Acadia National Park,Mammal,Carnivora,Canidae,Canis lupus,"Eastern Timber Wolf, Gray Wolf, Timber Wolf",Approved,Not Confirmed,Native,NaN,NaN,Endangered,NaN
4,ACAD-1004,Acadia National Park,Mammal,Carnivora,Canidae,Vulpes vulpes,"Black Fox, Cross Fox, Eastern Red Fox, Fox, Re...",Approved,Present,Unknown,Common,Breeder,NaN,NaN
5,ACAD-1005,Acadia National Park,Mammal,Carnivora,Felidae,Lynx canadensis,Canada Lynx,Approved,Not Confirmed,Native,NaN,NaN,NaN,NaN
6,ACAD-1006,Acadia National Park,Mammal,Carnivora,Felidae,Lynx rufus,"Bay Lynx, Bobcat, Red Lynx, Wild Cat",Approved,Not Present (Historical Report),Not Native,NaN,Vagrant,NaN,NaN
7,ACAD-1007,Acadia National Park,Mammal,Carnivora,Mephitidae,Mephitis mephitis,"Eastern Skunk, Polecat, Striped Skunk",Approved,Not Present (Historical Report),Native,NaN,NaN,NaN,NaN
8,ACAD-1008,Acadia National Park,Mammal,Carnivora,Mustelidae,Lutra canadensis,"Otter, River Otter",Approved,Present,Native,Common,Breeder,NaN,NaN
9,ACAD-1009,Acadia National Park,Mammal,Carnivora,Mustelidae,Martes pennanti,"Blackcat, Fisher, Pekan",Approved,Present,Native,Rare,NaN,NaN,NaN


In [5]:
df_pr.columns

Index(['Park Code', 'Park Name', 'State', 'Acres', 'Latitude', 'Longitude'], dtype='object')

In [6]:
df_pr.rename(columns={'Park Code':'park_code','Park Name':'park_name','State':'state','Acres':'acres','Latitude':'latitude','Longitude':'longitude'}, inplace=True)

In [7]:
df_pr.columns

Index(['park_code', 'park_name', 'state', 'acres', 'latitude', 'longitude'], dtype='object')

In [8]:
df_sp.columns

Index(['Species ID', 'Park Name', 'Category', 'Order', 'Family',
       'Scientific Name', 'Common Names', 'Record Status', 'Occurrence',
       'Nativeness', 'Abundance', 'Seasonality', 'Conservation Status',
       'Unnamed: 13'],
      dtype='object')

In [9]:
df_sp.drop(['Unnamed: 13'], axis=1, inplace=True)

In [10]:
df_sp.rename(columns={'Species ID':'species_ID','Park Name':'park_name','Category':'category','Order':'order','Family':'family','Scientific Name':'scientific_name','Common Names':'common_names','Record Status':'record_status','Occurence':'occurence','Nativeness':'nativeness','Abundance':'abundance','Seasonability':'seasonability','Conservation Status':'conservation_status'}, inplace=True)

In [11]:
df_sp.drop(['species_ID','scientific_name','record_status','Occurrence',
       'nativeness', 'abundance', 'Seasonality', 'conservation_status'], axis=1, inplace=True)

In [12]:
df_sp.columns

Index(['park_name', 'category', 'order', 'family', 'common_names'], dtype='object')

In [13]:
df_sp.head()

,park_name,category,order,family,common_names
0,Acadia National Park,Mammal,Artiodactyla,Cervidae,Moose
1,Acadia National Park,Mammal,Artiodactyla,Cervidae,"Northern White-Tailed Deer, Virginia Deer, Whi..."
2,Acadia National Park,Mammal,Carnivora,Canidae,"Coyote, Eastern Coyote"
3,Acadia National Park,Mammal,Carnivora,Canidae,"Eastern Timber Wolf, Gray Wolf, Timber Wolf"
4,Acadia National Park,Mammal,Carnivora,Canidae,"Black Fox, Cross Fox, Eastern Red Fox, Fox, Re..."


In [14]:
df_pr= df_pr.iloc[:,0:6]
df_pr.head()

,park_code,park_name,state,acres,latitude,longitude
0,ACAD,Acadia National Park,ME,47390,44.35,-68.21
1,ARCH,Arches National Park,UT,76519,38.68,-109.57
2,BADL,Badlands National Park,SD,242756,43.75,-102.50
3,BIBE,Big Bend National Park,TX,801163,29.25,-103.25
4,BISC,Biscayne National Park,FL,172924,25.65,-80.08


Creating a Data Frame and Merging

In [15]:
df_pr_sp = pd.merge(df_pr, df_sp)
df_pr_sp.head()

,park_code,park_name,state,acres,latitude,longitude,category,order,family,common_names
0,ACAD,Acadia National Park,ME,47390,44.35,-68.21,Mammal,Artiodactyla,Cervidae,Moose
1,ACAD,Acadia National Park,ME,47390,44.35,-68.21,Mammal,Artiodactyla,Cervidae,"Northern White-Tailed Deer, Virginia Deer, Whi..."
2,ACAD,Acadia National Park,ME,47390,44.35,-68.21,Mammal,Carnivora,Canidae,"Coyote, Eastern Coyote"
3,ACAD,Acadia National Park,ME,47390,44.35,-68.21,Mammal,Carnivora,Canidae,"Eastern Timber Wolf, Gray Wolf, Timber Wolf"
4,ACAD,Acadia National Park,ME,47390,44.35,-68.21,Mammal,Carnivora,Canidae,"Black Fox, Cross Fox, Eastern Red Fox, Fox, Re..."


In [16]:
df_pr_sp.drop(['park_code','order','family'], axis=1, inplace=True)

In [17]:
df_pr_sp.head()

,park_name,state,acres,latitude,longitude,category,common_names
0,Acadia National Park,ME,47390,44.35,-68.21,Mammal,Moose
1,Acadia National Park,ME,47390,44.35,-68.21,Mammal,"Northern White-Tailed Deer, Virginia Deer, Whi..."
2,Acadia National Park,ME,47390,44.35,-68.21,Mammal,"Coyote, Eastern Coyote"
3,Acadia National Park,ME,47390,44.35,-68.21,Mammal,"Eastern Timber Wolf, Gray Wolf, Timber Wolf"
4,Acadia National Park,ME,47390,44.35,-68.21,Mammal,"Black Fox, Cross Fox, Eastern Red Fox, Fox, Re..."


In [18]:
df_cat = df_pr_sp.groupby(['park_name', 'category'])[['category']].count()

In [21]:
df_cat = df_pr_sp.groupby(['park_name', 'category'])[['category']].count()

df_cat.to_csv("cat.csv", index=True)

In [22]:
cat_csv = pd.read_csv('cat.csv')

In [26]:
cat_csv.to_json(orient="records")

'[{"park_name":"Acadia National Park","category":"Amphibian","category.1":15},{"park_name":"Acadia National Park","category":"Bird","category.1":364},{"park_name":"Acadia National Park","category":"Fish","category.1":38},{"park_name":"Acadia National Park","category":"Mammal","category.1":55},{"park_name":"Acadia National Park","category":"Reptile","category.1":11},{"park_name":"Acadia National Park","category":"Vascular Plant","category.1":1226},{"park_name":"Arches National Park","category":"Amphibian","category.1":8},{"park_name":"Arches National Park","category":"Bird","category.1":205},{"park_name":"Arches National Park","category":"Fish","category.1":11},{"park_name":"Arches National Park","category":"Mammal","category.1":59},{"park_name":"Arches National Park","category":"Reptile","category.1":20},{"park_name":"Arches National Park","category":"Vascular Plant","category.1":745},{"park_name":"Badlands National Park","category":"Amphibian","category.1":10},{"park_name":"Badlands N